In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [110]:
MV = pd.read_csv('D:\Honzík\Škola\IES\V. Semestr\Data Processing in Python\Project\Market Values.csv')

In [111]:
PS = pd.read_csv('C:/Users/jancv/Final_Player_Table.csv')
PS = PS.drop_duplicates('player',keep=False)
PS = PS.rename(columns={'player':'Player'})

In [112]:
Dataset = pd.merge(PS,MV,on=['Player'],how='inner')
Dataset

,Player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,Foot,Market Value,Team_MV
0,Marko Alvir,CRO,MF,Viktoria Plzeň,26,1994,9,0,116,0,...,2,0,0.00,0.00,0.00,0.00,0.00,-,500,6400
1,Pavol Bajza,SVK,GK,Slovácko,29,1991,6,6,540,0,...,1,0,0.00,0.00,0.00,0.00,0.00,right,200,6600
2,David Bartek,CZE,MF,Bohemians 1905,32,1988,4,4,345,1,...,0,0,0.26,0.00,0.26,0.26,0.26,right,300,8675
3,Jean-David Beauguel,FRA,FW,Viktoria Plzeň,28,1992,14,5,604,6,...,0,0,0.89,0.15,1.04,0.75,0.89,right,1000,24600
4,Michal Beran,CZE,MF,Slovan Liberec,20,2000,10,7,610,0,...,0,0,0.00,0.30,0.30,0.00,0.30,-,300,58950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Jan Vondra,CZE,DF,Bohemians 1905,25,1995,9,6,550,0,...,2,0,0.00,0.16,0.16,0.00,0.16,-,250,8675
128,Nataniel Wybraniec,POL,MF,Opava,20,2000,1,0,4,0,...,0,0,0.00,0.00,0.00,0.00,0.00,both,50,5875
129,Jakub Yunis,CZE,FW,Sigma Olomouc,24,1996,4,3,211,2,...,0,0,0.85,0.00,0.85,0.85,0.85,right,200,7925
130,Jan Zidek,CZE,DF,Opava,36,1984,11,11,990,2,...,3,0,0.18,0.00,0.18,0.09,0.09,left,150,5875


Now, we will add the current ranking in the Fortuna League by teams.

In [113]:
#by 1.1.2021
ranks = {'Slavia Prague':1,'Jablonec':2,'Sparta Prague':3,'Sigma Olomouc':4,'FK Pardubice':5,'Slovácko':6,'Slovan Liberec':7
                        ,'Baník Ostrava':8,'Karviná':9,'Viktoria Plzeň':10,'Teplice':11,'Fastav Zlín':12,'České Budĕjov.':13
                        ,'Bohemians 1905':14,'Příbram':15,'Zbrojovka Brno':16,'Mladá Boleslav':17,'Opava':18}

In [114]:
s=[]
for k in Dataset['squad']:
    s.append(ranks[k])
Dataset['Team_rank'] = s

And further, we will scrape the latest FIFA nationality ranking and add it to the Dataset using dictionary.

In [115]:
url = 'https://www.fifa.com/fifa-world-ranking/ranking-table/men/' #by 10.12.2020 (latest)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
players = soup.find("table",{"id": "rank-table"}).tbody.find_all('tr')

keys = []
values = []
for idx,player in enumerate(players):
    keys.append(player.find("span",{"class":"fi-t__nTri"}).text)
    values.append(idx+1)
Nation_ranking = dict(zip(keys,values))

In [125]:
nation = []
for nat in Dataset['nationality']:
    try:
        nation.append(int(Nation_ranking[nat]))
    except:
        nation.append(0)

Dataset['Nation_rank'] = nation
Dataset = Dataset[Dataset['Nation_rank']!=0]

#### Renaming the positions

Sicne there are some players who played on more than one position, we need to specify which is the dominant one to have only
3 levels. This is done in the following step.

In [117]:
pos2 = []
for p in Dataset['position']:
    if len(p) == 2:
        pos2.append(p)
    else:
        pos2.append(p[0:2])
Dataset['Pos2'] = pos2

In [118]:
d_pos = {'DF':'Defender','MF':'Midfielder','FW':'Forward'}
Position = []
Dataset = Dataset[Dataset['Pos2']!='GK']
for po in Dataset['Pos2']:
    Position.append(d_pos[po])
Dataset['Position'] = Position

C:\Users\jancv\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [119]:
Dataset = Dataset.drop(columns=['position','Pos2'])

In [127]:
Dataset.columns

Index(['Player', 'nationality', 'squad', 'age', 'birth_year', 'games',
       'games_starts', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att',
       'cards_yellow', 'cards_red', 'goals_per90', 'assists_per90',
       'goals_assists_per90', 'goals_pens_per90', 'goals_assists_pens_per90',
       'Foot', 'Market Value', 'Team_MV', 'Team_rank', 'Nation_rank',
       'Position'],
      dtype='object')

### OLS

First of all, we would like to say that since we have a lot of variables and comparatively low amount of observations,
the OLS is not ideal. This project is, however, more of a data science one (i.e. getting the data, merging it etc.)
than econometric one (checking the validity of assumptions.)

In [49]:
import statsmodels.api as sm

In [121]:
Y = Dataset['Market Value']
age = Dataset['age']
goals = Dataset['goals']
games = Dataset['games']
assists = Dataset['assists']
tr = Dataset['Team_rank']
tmv = Dataset['Team_MV']
nr = Dataset['Nation_rank']


X = np.column_stack((goals,assists,tmv))
X = sm.add_constant(X)

model = sm.OLS(Y,X)
X = sm.add_constant(X)

In [122]:
fit = model.fit()

In [123]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Market Value   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     37.22
Date:                Thu, 28 Jan 2021   Prob (F-statistic):           5.79e-17
Time:                        11:16:52   Log-Likelihood:                -972.18
No. Observations:                 121   AIC:                             1952.
Df Residuals:                     117   BIC:                             1964.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -160.9902    104.926     -1.534      0.128    -368.791      46.810
x1           210.2719     49.370      4.259      0.000     112.497     308.047
x2            68.3514     76.179      0.897      0.371     -82.518     219.220
x3             0.0334      0.004      8.309      0.000       0.025       0.041
==============================================================================
Omnibus:                       99.972   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1281.416
Skew:                           2.643   Prob(JB):                    5.55e-279
Kurtosis:                      18.040   Cond. No.                     3.79e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

As we can see the model behaves badly. If we were to collect more observations, it would definitely improve.
